In [1]:
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import tensorflow as tf
import numpy as np

In [2]:
model = load_model("./models/mask_classificatioon.h5")
face_cascad_model = cv2.CascadeClassifier("./models/haarcascade_frontalface_default.xml")

In [4]:
video = cv2.VideoCapture(0)

while True:
    rat, frame = video.read()
    
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascad_model.detectMultiScale(img_gray, 1.3, 4, minSize=(10, 10))
    
    if len(faces) != 0:
        for x,y,w,h in faces:            
            yuz_rasmi = frame[y:y+h, x:x+w, :]
            yuz_rasmi = cv2.resize(yuz_rasmi,(224, 224), interpolation=cv2.INTER_CUBIC)
            yuz_rasmi = preprocess_input(yuz_rasmi)
            result = model.predict(np.expand_dims(yuz_rasmi, 0), verbose=0)

            if np.round(result,2) > 0.50:   # np.round(result, 2) > 0.5
                maska_statusi = 'Maska bor'                
            else:
                maska_statusi = 'Maska yo\'q' 
            frame = cv2.putText(frame, maska_statusi, (x,y-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
            
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)
    else:
        maska_statusi = "Yuz aniqlanmadi !"
            
    frame = cv2.putText(frame, maska_statusi, (x,y-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    cv2.imshow("face", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video.release()
cv2.destroyAllWindows()